In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

train_data = pd.read_csv('Data/train.csv')
test_data = pd.read_csv("Data/test.csv")

In [4]:
category = train_data['category']

set_category = set()
for c in category:
    c_arr = c.split(',')
    c_arr = [x.strip() for x in c_arr if x != '' and x != '-']
    set_category.update(c_arr)
print("{} categories".format(len(set_category)))
print(set_category)



15 categories
{'coe car', 'consignment car', 'low mileage car', 'vintage cars', 'almost new car', 'premium ad car', 'rare & exotic', 'sta evaluated car', 'sgcarmart warranty cars', 'parf car', 'imported used vehicle', 'hybrid cars', 'direct owner sale', 'electric cars', 'opc car'}
